## Import statements

In [33]:
import pandas as pd

## Importing the data

Underlying data to find the coordinates is a CSV file, downloaded [here](https://simplemaps.com/data/canada-cities).
The Excel "myplaces" is a list of places I visited, only including the place the name, no coordinates or provinces.

In [34]:
cities_data = pd.read_csv("canadacities.csv")
cities_data.head()

,city,city_ascii,province_id,province_name,lat,lng,population,density,timezone,ranking,postal,id
0,Toronto,Toronto,ON,Ontario,43.7417,-79.3733,5647656.0,4427.8,America/Toronto,1,M5T M5V M5P M5S M5R M5E M5G M5A M5C M5B M5M M5...,1124279679
1,Montréal,Montreal,QC,Quebec,45.5089,-73.5617,3675219.0,4833.5,America/Toronto,1,H1X H1Y H1Z H1P H1R H1S H1T H1V H1W H1H H1J H1...,1124586170
2,Vancouver,Vancouver,BC,British Columbia,49.2500,-123.1000,2426160.0,5749.9,America/Vancouver,1,V6Z V6S V6R V6P V6N V6M V6L V6K V6J V6H V6G V6...,1124825478
3,Calgary,Calgary,AB,Alberta,51.0500,-114.0667,1306784.0,1592.4,America/Edmonton,1,T1Y T2H T2K T2J T2M T2L T2N T2A T2C T2B T2E T2...,1124690423
4,Edmonton,Edmonton,AB,Alberta,53.5344,-113.4903,1151635.0,1320.4,America/Edmonton,1,T5X T5Y T5Z T5P T5R T5S T5T T5V T5W T5H T5J T5...,1124290735


In [35]:
# only working with relevant data
cities_df = cities_data[["city", "province_id", "lat", "lng"]]
cities_df.head()

,city,province_id,lat,lng
0,Toronto,ON,43.7417,-79.3733
1,Montréal,QC,45.5089,-73.5617
2,Vancouver,BC,49.2500,-123.1000
3,Calgary,AB,51.0500,-114.0667
4,Edmonton,AB,53.5344,-113.4903


In [36]:
places_df = pd.read_excel("myplaces.xlsx", header=None)
places_df.columns = ["city"]
places_df.head()

,city
0,Airdrie
1,Campbell River
2,Barrie
3,Athabasca
4,Banff


In [37]:
# checking data, no null values, correct datatypes
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1737 entries, 0 to 1736
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         1737 non-null   object 
 1   province_id  1737 non-null   object 
 2   lat          1737 non-null   float64
 3   lng          1737 non-null   float64
dtypes: float64(2), object(2)
memory usage: 54.4+ KB


In [38]:
places_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   city    89 non-null     object
dtypes: object(1)
memory usage: 844.0+ bytes


In [39]:
# checking for duplicates
print(cities_df[cities_df.duplicated()==True])

Empty DataFrame
Columns: [city, province_id, lat, lng]
Index: []


In [40]:
print(places_df[places_df.duplicated()==True])

               city
75  Bruce Peninsula


In [41]:
places_df.drop_duplicates(keep="first", inplace=True)

In [42]:
print(places_df[places_df.duplicated()==True])

Empty DataFrame
Columns: [city]
Index: []


## Matching the Data

In [50]:
merged_df = pd.merge(places_df, cities_df, how='inner', on='city')

In [51]:
merged_df.head()

,city,province_id,lat,lng
0,Airdrie,AB,51.2917,-114.0144
1,Campbell River,BC,50.0244,-125.2475
2,Barrie,ON,44.3711,-79.6769
3,Athabasca,AB,54.7197,-113.2856
4,Banff,AB,51.1781,-115.5719


In [52]:
merged_df.shape

(63, 4)

The merged_df only contains 63 cities, while places_df. The underlying CSV only covers places with more than 1000 residents, so Provincial Parks or Ski Resorts are not listed, I will deal with those places in the future. 

Another problem might be, that some cities might be spelled slighlty different, this will be something to address in the future as well (though: using SQL DBs instead as merging on similar entries will be easier)

## Checking the newly created Data Frame

In [59]:
merged_df["province_id"].unique()

array(['AB', 'BC', 'ON', 'QC', 'NL', 'NB', 'SK'], dtype=object)

I have only been to the provinces Alberta, British Columbia, Sasketchwan, Manitoba, Ontario and Quebec.
So "NL" and "NB" shouldn't be in there. Let's check why why they are...

In [69]:
merged_df[merged_df["province_id"]=="NL"]

,city,province_id,lat,lng
51,Victoria,NL,47.7675,-53.2411


In [70]:
merged_df[merged_df["province_id"]=="NB"]

,city,province_id,lat,lng
58,Southampton,NB,46.0789,-67.3124


I have been to Victoria, BC and Southhampton, ON. But city names might be the same in other provinces. So I have to drop these values and also double check the other provinces in case there are more places like this.

In [72]:
merged_df.drop([51, 58], inplace=True)

In [46]:
merged_df[merged_df["province_id"]=="AB"]

,city,province_id,lat,lng
0,Airdrie,AB,51.2917,-114.0144
3,Athabasca,AB,54.7197,-113.2856
4,Banff,AB,51.1781,-115.5719
9,Calgary,AB,51.0500,-114.0667
10,Canmore,AB,51.0890,-115.3590
18,Cochrane,AB,51.1890,-114.4670
23,Drumheller,AB,51.4636,-112.7194
32,Jasper,AB,52.9013,-118.1312
38,Medicine Hat,AB,50.0417,-110.6775


In [47]:
merged_df[merged_df["province_id"]=="BC"]

,city,province_id,lat,lng
1,Campbell River,BC,50.0244,-125.2475
5,Comox,BC,49.6733,-124.9022
8,Fernie,BC,49.5042,-115.0628
11,Golden,BC,51.3019,-116.9667
13,Invermere,BC,50.5083,-116.0303
15,Kamloops,BC,50.6761,-120.3408
16,Kelowna,BC,49.8881,-119.4956
17,Kimberley,BC,49.6697,-115.9775
26,Lillooet,BC,50.6864,-121.9364
29,Nanaimo,BC,49.1642,-123.9364


In [48]:
merged_df[merged_df["province_id"]=="SK"]

,city,province_id,lat,lng
62,Regina,SK,50.4547,-104.6067


In [49]:
merged_df[merged_df["province_id"]=="MB"]

,city,province_id,lat,lng
52,Victoria,MB,49.6644,-98.9153


I have not been to Victoria in Manitoba (but BC), so this entry can go

In [55]:
merged_df.drop([52], inplace=True)

In [57]:
merged_df[merged_df["province_id"]=="ON"]

,city,province_id,lat,lng
2,Barrie,ON,44.3711,-79.6769
7,Collingwood,ON,44.5000,-80.2167
12,Guelph,ON,43.5500,-80.2500
14,Hamilton,ON,43.2567,-79.8692
19,Cochrane,ON,49.0667,-81.0167
22,Kenora,ON,49.7667,-94.4833
24,Marathon,ON,48.7500,-86.3667
25,Muskoka Falls,ON,45.1264,-79.5580
27,Niagara Falls,ON,43.0600,-79.1067
28,Niagara-on-the-Lake,ON,43.2553,-79.0717


In [58]:
merged_df[merged_df["province_id"]=="QC"]

,city,province_id,lat,lng
6,Carleton-sur-Mer,QC,48.1000,-66.1333
20,Frontenac,QC,45.5800,-70.8300
21,Gaspé,QC,48.8333,-64.4833
33,Montréal,QC,45.5089,-73.5617
34,Mont-Royal,QC,45.5161,-73.6431
35,Mont-Tremblant,QC,46.1167,-74.6000
40,Quebec City,QC,46.8139,-71.2081
59,Rivière-du-Loup,QC,47.8333,-69.5333
60,Rimouski,QC,48.4500,-68.5300
61,Percé,QC,48.5333,-64.2167


Now the data should be clean and I can move on to the next step of creating the map. I save the df to a file so I can resuse it in a new Notebook for easier documentation.

In [74]:
merged_df.to_csv('cities_cleaned.csv')